<a href="https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/NeuroCensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">NeuroCensor v.1.0</font></b>

![visitors](https://visitor-badge.glitch.me/badge?page_id=mlart.ТeuralСensor&left_color=black&right_color=orange)

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [MTTR](https://github.com/mttr2021/MTTR), [LaMa](https://github.com/saic-mdal/lama)

Creators: **[mttr2021](https://github.com/mttr2021), [saic-mdal](https://github.com/saic-mdal)**

Colab based on mttr2021's [notebook](https://colab.research.google.com/drive/12p0jpSx3pJNfZk-y_L44yeHZlhsKVra-).

<b><font color="black" size="+2">Colab created by:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@MLArt](https://t.me/MLArt),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

---


```
(ENG) To get started, click on the button (where the red arrow indicates). After clicking, wait until the execution is complete.
```
```
(RUS) Чтобы начать, поочередно нажимайте на кнопки (куда указывают красные стрелки), дожидаясь завершения выполнения каждого блока.
```


In [1]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Install all necessary libraries</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Установить все необходимые библиотеки</font></b>

try: 
  !pip3 install googletrans==3.1.0a0
  from googletrans import Translator, constants
  from pprint import pprint
  translator = Translator()
except: pass

!pip install av moviepy yt-dlp ruamel.yaml einops timm transformers
import cv2
import os
import glob
import torch
import torchvision
import torchvision.transforms.functional as F
from einops import rearrange
import numpy as np
from PIL import Image, ImageDraw, ImageOps, ImageFont
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from google.colab import files
from IPython.display import clear_output
from IPython.display import HTML
from base64 import b64encode
from tqdm.notebook import trange, tqdm
from transformers import logging
logging.set_verbosity_error()

model, postprocessor = torch.hub.load('mttr2021/MTTR:main','mttr_refer_youtube_vos', force_reload=True)
model = model.cuda()

class NestedTensor(object):
    def __init__(self, tensors, mask):
        self.tensors = tensors
        self.mask = mask

def nested_tensor_from_videos_list(videos_list):
    def _max_by_axis(the_list):
      maxes = the_list[0]
      for sublist in the_list[1:]:
          for index, item in enumerate(sublist):
              maxes[index] = max(maxes[index], item)
      return maxes

    max_size = _max_by_axis([list(img.shape) for img in videos_list])
    padded_batch_shape = [len(videos_list)] + max_size
    b, t, c, h, w = padded_batch_shape
    dtype = videos_list[0].dtype
    device = videos_list[0].device
    padded_videos = torch.zeros(padded_batch_shape, dtype=dtype, device=device)
    videos_pad_masks = torch.ones((b, t, h, w), dtype=torch.bool, device=device)
    for vid_frames, pad_vid_frames, vid_pad_m in zip(videos_list, padded_videos, videos_pad_masks):
        pad_vid_frames[:vid_frames.shape[0], :, :vid_frames.shape[2], :vid_frames.shape[3]].copy_(vid_frames)
        vid_pad_m[:vid_frames.shape[0], :vid_frames.shape[2], :vid_frames.shape[3]] = False
    return NestedTensor(padded_videos.transpose(0, 1), videos_pad_masks.transpose(0, 1))

!git clone https://github.com/saic-mdal/lama.git lama
%cd /content/lama
!pip install -r requirements.txt --quiet
!pip install wget --quiet
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
!unzip big-lama.zip
%cd /content
!mkdir frames result

if os.path.isdir("/content/result"):
  clear_output()
  print('Done!')

In [48]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload video</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Загрузить видео</font></b>

%cd /content
video_url = "https://www.youtube.com/watch?v=K1ZFaayCJ8A" #@param {type:"string"}
download_resolution = "720" #@param ["360", "480", "720", "1080"]
download_resolution = int(download_resolution)
#@markdown or 
upload_from_your_local_system = False #@param {type:"boolean"}
start_trimming = 23 #@param {type:"integer"}
duration_trimming = 6 #@param {type:"slider", min:1, max:20, step:1}

input_clip_path = '/content/input_clip.mp4'

if upload_from_your_local_system == False:
  ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': '/content/full_video.mp4'}
  with YoutubeDL(ydl_opts) as ydl:
      ydl.download([video_url])
  with VideoFileClip('/content/full_video.mp4') as video:
      subclip = video.subclip(start_trimming, start_trimming+duration_trimming)
      subclip.write_videofile(input_clip_path)
else:
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  !ffmpeg -y -ss $start_trimming -i $fn -to $duration_trimming -c:v copy -c:a copy -async 1 $input_clip_path

!ffmpeg -y -i $input_clip_path -vn -acodec copy /content/audio.aac -hide_banner -loglevel error

clear_output()
input_clip = open(input_clip_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(input_clip).decode()
HTML("""<video width=480 controls><source src="%s" type="video/mp4"></video>""" % data_url)

In [43]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Start censoring</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Запустить цензурирование</font></b>

#@markdown (ENG) If you need to delete several objects, separate them with the "|" (without quotes)

#@markdown (RUS) Если нужно удалить несколько объектов, то разделите их символом "|" (без ковычек)
text = "Grizzly bear | \u041C\u0443\u0436\u0447\u0438\u043D\u0430 \u0432 \u043A\u0443\u0440\u0442\u043A\u0435" #@param {type:"string"}
try:
  from googletrans import Translator, constants
  from pprint import pprint
  translator = Translator()
  translation = translator.translate(text)
  text_queries = translation.text
except: text_queries = text

#@markdown ---
additional_mask_thickness = 10 #@param {type:"integer"}
window_length = 24 #@param {type:"integer"}
window_overlap = 6 #@param {type:"integer"}

%cd /content
!rm -rf /content/frames/*
with torch.inference_mode():
  # read and preprocess the video clip:
  video, audio, meta = torchvision.io.read_video(filename=input_clip_path)
  fps_of_video = meta['video_fps']
  video = rearrange(video, 't h w c -> t c h w')
  input_video = F.resize(video, size=360, max_size=640).cuda()
  input_video = input_video.to(torch.float).div_(255)
  input_video = F.normalize(input_video, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  video_metadata = {'resized_frame_size': input_video.shape[-2:], 'original_frame_size': video.shape[-2:]}
  
  # partition the clip into overlapping windows of frames:
  windows = [input_video[i:i+window_length] for i in range(0, len(input_video), window_length - window_overlap)]
  # clean up the text queries:
  text_queries = [" ".join(q.lower().split()) for q in text_queries.split('|')]

  pred_masks_per_query = []
  t, _, h, w = video.shape
  for text_query in tqdm(text_queries, desc='text queries'):
    pred_masks = torch.zeros(size=(t, 1, h, w))
    for i, window in enumerate(tqdm(windows, desc='windows')):
      window = nested_tensor_from_videos_list([window])
      valid_indices = torch.arange(len(window.tensors)).cuda()
      outputs = model(window, valid_indices, [text_query])
      window_masks = postprocessor(outputs, [video_metadata], window.tensors.shape[-2:])[0]['pred_masks']
      win_start_idx = i*(window_length-window_overlap)
      pred_masks[win_start_idx:win_start_idx + window_length] = window_masks
    pred_masks_per_query.append(pred_masks)

video_np = rearrange(video, 't c h w -> t h w c').numpy() / 255.0
pred_masks_per_frame = rearrange(torch.stack(pred_masks_per_query), 'q t 1 h w -> t q h w').numpy()
for vid_frame, frame_masks in tqdm(zip(video_np, pred_masks_per_frame), total=len(video_np), desc='applying masks...'):
  name = len(glob.glob('/content/frames/*.*g')) // 2
  cur_frame = vid_frame * 255
  cur_frame = cv2.cvtColor(cur_frame.astype(np.uint8), cv2.COLOR_BGR2RGB)
  cv2.imwrite(f'/content/frames/{str(name).zfill(7)}.png', cur_frame)
  mask = np.zeros((h, w),dtype="float32")
  for inst_mask in frame_masks:
    logic_mask = np.logical_or(mask, inst_mask)
    mask = np.where(logic_mask == True, 1, 0)
  mask = mask.astype(np.uint8) * 255
  contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  cv2.drawContours(mask, contours, -1, (255,255,255), additional_mask_thickness)
  final_mask = np.repeat(mask[..., np.newaxis], 3, axis=2)
  cv2.imwrite(f'/content/frames/{str(name).zfill(7)}_mask.png', final_mask)

%cd /content/lama
!rm -rf /content/result/*
print('start deleting...')
!PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=/content/lama/big-lama indir=/content/frames outdir=/content/result dataset.img_suffix=.png > /dev/null device=cuda

!ffmpeg -y -r $fps_of_video -i /content/result/%7d_mask.png -c:v libx264 -vf fps=$fps_of_video -pix_fmt yuv420p /content/output_clip_without_audio.mp4 -hide_banner -loglevel error

if os.path.isfile('/content/output_clip_without_audio.mp4'):
  clear_output()
  print('Done!')

Done!


In [45]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Show result</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Показать результат</font></b>

input_clip = open('/content/output_clip_without_audio.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(input_clip).decode()
HTML("""<video width=480 controls><source src="%s" type="video/mp4"></video>""" % data_url)

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download result</font></b>
#@markdown <b><font color="black" size="+1"> (RUS) Скачать результат</font></b>

try:
  !ffmpeg -y -i /content/output_clip_without_audio.mp4 -i /content/audio.aac /content/output_clip.mp4 -hide_banner -loglevel error
  files.download('/content/output_clip.mp4')
except: files.download('/content/output_clip_without_audio.mp4')